In [ ]:
#Alur
#Setup
#Buka File
#Lihat data
#Tentukan X dan y
#Cek missing value
#Handle missing value
#Feature Engineering
#Buat Model
#Cari RMSE paling kecil
#fit model ke data test
#Finishing

In [1]:
#Setup
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

print("Setup Done")

Setup Done


In [17]:
#Import Data
data = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/train.csv")
tdata = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/test.csv")
print(data.shape)
print(data.columns)

(630000, 13)
Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')


In [3]:
#Lihat data
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [ ]:
mis = data.isnull().sum()
mis[0:13]

In [24]:
#cari kolom kategorikal untuk model catboost

for col in data:
    if data[col].dtype == "object":
        print(col)

cat_col= [
    'sleep_quality',
    'study_method',
    'facility_rating',
    'internet_access'
]

gender
course
internet_access
sleep_quality
study_method
facility_rating
exam_difficulty


In [25]:
#tentukan X dan y
y = data["exam_score"]
X = data.drop(["id","exam_score","gender","course", 'exam_difficulty'], axis = 1)
# X = data.drop(["id","exam_score","internet_access","gender","exam_difficulty"], axis = 1)

In [14]:
#factorize category type columns
for col in X.select_dtypes("object"):
    X[col], _ = X[col].factorize()

fcol = X.dtypes ==int

In [16]:
#mi score
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X,y, cat_col):
    mi_scores = mutual_info_regression(X, y, discrete_features= fcol)
    mi_scores = pd.Series(mi_scores, name= "MI Scores", index = X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X,y,fcol)
mi_scores[::]

study_hours         0.913950
class_attendance    0.301945
sleep_hours         0.073860
study_method        0.040896
sleep_quality       0.036648
facility_rating     0.030615
age                 0.011017
exam_difficulty     0.008920
internet_access     0.001987
Name: MI Scores, dtype: float64

In [26]:
#buat model
model = CatBoostRegressor(
    iterations=1500, 
    learning_rate= 0.05,
    depth = 6,
    loss_function= 'RMSE',
    cat_features= cat_col,
    random_seed= 2)

In [27]:
#Train model
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size= 0.2, random_state=1)
model.fit(train_X,train_y,)

0:	learn: 18.3136215	total: 155ms	remaining: 3m 52s
1:	learn: 17.7359709	total: 287ms	remaining: 3m 35s
2:	learn: 17.1959494	total: 395ms	remaining: 3m 17s
3:	learn: 16.6894788	total: 524ms	remaining: 3m 16s
4:	learn: 16.2147473	total: 636ms	remaining: 3m 10s
5:	learn: 15.7764288	total: 753ms	remaining: 3m 7s
6:	learn: 15.3642926	total: 857ms	remaining: 3m 2s
7:	learn: 14.9723022	total: 987ms	remaining: 3m 4s
8:	learn: 14.6102926	total: 1.1s	remaining: 3m 2s
9:	learn: 14.2722155	total: 1.21s	remaining: 3m
10:	learn: 13.9516744	total: 1.33s	remaining: 3m
11:	learn: 13.6602836	total: 1.44s	remaining: 2m 59s
12:	learn: 13.3860281	total: 1.56s	remaining: 2m 58s
13:	learn: 13.1192704	total: 1.69s	remaining: 2m 59s
14:	learn: 12.8677933	total: 1.82s	remaining: 2m 59s
15:	learn: 12.6316610	total: 1.93s	remaining: 2m 59s
16:	learn: 12.4100846	total: 2.06s	remaining: 2m 59s
17:	learn: 12.2061125	total: 2.19s	remaining: 3m
18:	learn: 12.0164749	total: 2.34s	remaining: 3m 2s
19:	learn: 11.8406327

In [28]:
#predict
preds_vy= model.predict(val_X) #predict jawaban ujian
preds_ty= model.predict(train_X) #predict jawaban latihan soal

In [29]:
#Model Validation
train_rmse = root_mean_squared_error(train_y, preds_ty)
val_rmse = root_mean_squared_error(val_y,preds_vy)
print(train_rmse)
print(val_rmse)

8.732803425614126
8.772424148139098


In [ ]:
penting = model.get_feature_importance(prettified=True)
print(penting)

In [ ]:
#fit model ke Semua data
model.fit(X,y)

In [ ]:
#Prediksi test data
tdata_fit= tdata.drop(["id","gender","course"], axis=1)
pred_test= model.predict(tdata_fit)

In [ ]:
print(X.columns)
print(tdata_fit.columns)

In [ ]:
#Buat ke dataset
submit = pd.DataFrame({"id":tdata.id, "exam_score": pred_test})
#Buat dataset ke file
submit.to_csv("P1.csv", index= False)